In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cnv_df 
: CCLE copy number data. values are log2 copy number ratio. columns are 34 breast cancer cell lines. rows are genes.

cyclops 
: numpy array of 133 cyclops genes from project DRIVE

ess_df 
: essentiality data. Columns are 77 breast cancer cell lines. Rows are genes. Values 1 for essential at c1 cutoff (z-score -2). -1 for non-essential c1 cutoff. 0 can be also considered as non-essential.

In [13]:
cnv_df = pd.read_table('../CCLE_BR_lines_CNV.txt', header=0, index_col=0, sep='\t')
# parse out '_BREAST' part in cell line names
cnv_df.columns = [token[0] for token in cnv_df.columns.str.split('_')]
cyclops = pd.read_table('../CYCLOPS_project_drive.txt', header=None, names=['gene'], index_col=False)
cyclops = cyclops.gene.values
ess_df = pd.read_table('../breast_essential_c1.mat', header=0, index_col=0, sep='\t')
ess_df.columns = ess_df.columns.str.upper()

# drop cell lines (columns) that are not in the 77 cell lines
cnv_df.drop(list(set(cnv_df.columns) - set(ess_df.columns)), axis=1, inplace=True)

In [14]:
# leave only CYCLOPS gene
cnv_cyclops_df = cnv_df.loc[cyclops, :]
ess_cyclops_df = ess_df.loc[cyclops, :]

In [39]:
# construct new dataframe
cnt_df = pd.DataFrame(index= cnv_cyclops_df.index, data = 0,
                      columns=['ess_part_loss', 'part_loss', 'ess_homo_loss', 'homo_loss', 'ess_neutral', 'neutral'])

In [40]:
# consider -1.28 <= x < -0.1 as partial loss

for idx, row in cnv_cyclops_df.iterrows():
    for col in cnv_cyclops_df.columns:
        label = ''
        cnv = row[col]
        if (-1.28 <= cnv) and (cnv < -0.1): # if partial loss
            label = 'part_loss'
        elif cnv < -1.28: # homologous loss
            label = 'homo_loss'
        else: # neutral
            label = 'neutral'
        ### END - if else        
     
        cnt_df.at[idx, label] += 1
        if ess_cyclops_df.loc[idx, col] == 1: # if essential
            cnt_df.at[idx, 'ess_'+label] += 1
    ### END - for col
### END - for idx, row

In [45]:
cnt_df = cnt_df[(cnt_df['part_loss'] >= 3) & (cnt_df['neutral'] >= 3)]

In [62]:
cyc_list2 = []
for idx, row in cnt_df.iterrows():
    a = float(row['ess_part_loss'])/row['part_loss']
    b = float(row['ess_neutral'])/row['neutral']
    if a > 0.7 and a - b > 0.2:
        print '{0}\t{1}\t{2}'.format(idx, a, b)
        cyc_list2.append(idx)
len(cyc_list2)

PSMA4	0.75	0.2
RPA2	0.95	0.545454545455
SRSF1	1.0	0.777777777778
CWC22	0.888888888889	0.454545454545
SRSF3	1.0	0.461538461538
PSMC1	0.857142857143	0.375
PSMC2	1.0	0.608695652174
AURKB	0.833333333333	0.615384615385
DDX46	0.9	0.619047619048
PSMC3	0.888888888889	0.5
PSMC5	0.727272727273	0.4
PSMD12	1.0	0.608695652174
EIF2S2	1.0	0.75
PLRG1	0.8125	0.6
EIF3G	0.928571428571	0.705882352941
EIF3I	1.0	0.666666666667
POLR2A	0.833333333333	0.615384615385
SF3A1	1.0	0.733333333333
EIF4A3	1.0	0.695652173913
SF3B4	1.0	0.28
PPWD1	0.705882352941	0.5
COPB2	0.714285714286	0.291666666667
COPS6	0.714285714286	0.458333333333
XAB2	0.736842105263	0.416666666667


24

In [65]:
cnt_df['ess/part_loss'] = 0
cnt_df['ess/neutral'] = 0

for idx, row in cnt_df.iterrows():
    a = float(row['ess_part_loss'])/row['part_loss']
    b = float(row['ess_neutral'])/row['neutral']
    cnt_df.loc[idx, 'ess/part_loss'] = a
    cnt_df.loc[idx, 'ess/neutral'] = b

/home/hwangbo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/hwangbo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/hwangbo/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [67]:
cnt_df.to_csv('../cyclops_essential_partial_loss_ratio', sep='\t')

In [50]:
breast_cyclops = []
with open('../cyclops_tcga_ccle_del.txt', 'r') as f:
    for line in f.readlines():
        breast_cyclops.append(line.strip())

In [57]:
len(set(breast_cyclops) & set(cyc_list))

10

In [54]:
cnt_df

,ess_part_loss,part_loss,ess_homo_loss,homo_loss,ess_neutral,neutral
gene,,,,,,
ABCE1,9,16,0,0,8,15
CTR9,7,19,0,0,2,12
MCRS1,2,14,0,0,0,17
PSMA4,12,16,0,0,3,15
RPA2,19,20,0,0,6,11
SRSF1,13,13,0,0,14,18
ALYREF,9,10,0,0,18,21
CWC22,8,9,0,0,10,22
MED11,0,18,0,0,0,13
